## Importing Necessary Libraries

We will try to find if a formula can be estimated using data to calculate CIBIL score

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os
import time

In [3]:
# Load the dataset
a1 = pd.read_excel(r"C:\Users\krish\OneDrive\Desktop\PBL\Internal_Bank_Dataset.xlsx")
a2 = pd.read_excel(r"C:\Users\krish\OneDrive\Desktop\PBL\External_Cibil_Dataset.xlsx")

In [4]:
df1 = a1.copy()
df2 = a2.copy()

In [20]:
print(df1)

       PROSPECTID  Total_TL  Tot_Closed_TL  Tot_Active_TL  \
0               1         5              4              1   
1               2         1              0              1   
2               3         8              0              8   
3               4         1              0              1   
4               5         3              2              1   
...           ...       ...            ...            ...   
51331       51332         3              0              3   
51332       51333         4              2              2   
51333       51334         2              1              1   
51334       51335         2              1              1   
51335       51336         1              0              1   

       Total_TL_opened_L6M  Tot_TL_closed_L6M  pct_tl_open_L6M  \
0                        0                  0            0.000   
1                        0                  0            0.000   
2                        1                  0            0.125   
3  

In [6]:
# # Remove nulls
# df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]

In [7]:
df2

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,15,24,23,2,24,24,0,0,0,...,0.0,0.0,0.0,1.661,0,0,ConsumerLoan,ConsumerLoan,650,P4
51332,51333,57,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.520,0,0,others,others,702,P1
51333,51334,32,-99999,-99999,0,-99999,0,0,0,0,...,0.0,1.0,0.0,0.567,0,0,ConsumerLoan,others,661,P3
51334,51335,58,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,1.202,0,0,ConsumerLoan,others,686,P2


In [8]:
# Merge the two dataframes, inner join so that no nulls are present
df = pd. merge ( df1, df2, how ='inner', left_on = ['PROSPECTID'], right_on = ['PROSPECTID'] )

In [9]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,1,0,1,1,0,1.000,0.00,1.000,0.000,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.0,1.661,0,0,ConsumerLoan,ConsumerLoan,650,P4
51332,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.0,0.520,0,0,others,others,702,P1
51333,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,0.0,1.0,0.0,0.567,0,0,ConsumerLoan,others,661,P3
51334,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.0,1.202,0,0,ConsumerLoan,others,686,P2


In [10]:
df = df[~df.isin([-99999]).any(axis=1)]

In [11]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
45,46,12,7,5,7,4,0.583,0.333,0.417,0.583,...,1.0,0.615,1.000,7.729,0,0,CC,others,651,P4
183,184,9,5,4,0,0,0.000,0.000,0.444,0.556,...,1.0,0.286,0.286,23.113,0,0,PL,others,672,P2
351,352,4,0,4,1,0,0.250,0.000,1.000,0.000,...,0.0,0.000,0.000,5.582,0,1,HL,CC,689,P2
403,404,129,122,7,0,7,0.000,0.054,0.054,0.946,...,1.0,0.500,0.429,2.218,1,0,ConsumerLoan,AL,649,P4
493,494,4,1,3,1,0,0.250,0.000,0.750,0.250,...,1.0,0.333,1.000,6.192,0,0,PL,PL,662,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51019,51020,23,17,6,0,0,0.000,0.000,0.261,0.739,...,0.0,0.000,0.000,22.434,0,1,PL,ConsumerLoan,742,P1
51044,51045,17,6,11,2,0,0.118,0.000,0.647,0.353,...,0.0,0.250,0.000,7.910,0,0,ConsumerLoan,ConsumerLoan,672,P2
51079,51080,19,13,6,0,1,0.000,0.053,0.316,0.684,...,0.0,0.000,0.000,29.221,1,0,PL,ConsumerLoan,688,P2
51126,51127,4,2,2,0,0,0.000,0.000,0.500,0.500,...,0.0,0.000,0.000,8.853,0,0,CC,PL,680,P2


In [12]:
# check how many columns are categorical
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score


In [14]:
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score']

In [15]:
X_encoded = pd.get_dummies(X, drop_first=True)  # Convert categorical columns to one-hot encoding

# Selecting the top 10 features
selector = SelectKBest(score_func=f_regression, k=min(7, X_encoded.shape[1]))
X_new = selector.fit_transform(X_encoded, y)

# Get the selected feature names
selected_features = X_encoded.columns[selector.get_support()]
print("Selected Features:", selected_features.tolist())

Selected Features: ['Age_Oldest_TL', 'num_std', 'num_std_6mts', 'num_std_12mts', 'enq_L6m', 'enq_L3m', 'Approved_Flag_P4']


In [16]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Running the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Model predictions
y_pred = model.predict(X_test)

# Formula for regression
coefficients = model.coef_
intercept = model.intercept_

print("\nRegression Formula:")
formula = f"Target = {intercept:.2f} " + " + ".join(
    [f"{coef:.2f}*{feature}" for coef, feature in zip(coefficients, selected_features)]
)
print(formula)

# Model performance
print("\nR^2 Score:", r2_score(y_test, y_pred))


Regression Formula:
Target = 666.71 0.22*Age_Oldest_TL + 0.03*num_std + 0.16*num_std_6mts + 0.79*num_std_12mts + 0.15*enq_L6m + -4.25*enq_L3m + -16.28*Approved_Flag_P4

R^2 Score: 0.8624685964749286


In [17]:
X_encoded[['Age_Oldest_TL', 'num_std', 'num_std_6mts', 'num_std_12mts', 'enq_L6m', 'enq_L3m', 'Approved_Flag_P4']]

,Age_Oldest_TL,num_std,num_std_6mts,num_std_12mts,enq_L6m,enq_L3m,Approved_Flag_P4
45,48,6,5,6,9,5,True
183,137,0,0,0,7,6,False
351,19,26,15,25,0,0,False
403,97,3,0,0,8,4,True
493,44,0,0,0,2,1,False
...,...,...,...,...,...,...,...
51019,206,34,13,21,0,0,False
51044,66,0,0,0,3,1,False
51079,63,0,0,0,0,0,False
51126,62,18,5,11,0,0,False


In [18]:
input_data = pd.DataFrame({
        'Age_Oldest_TL': [156],
        'num_std': [24],
        'num_std_6mts': [6],
        'num_std_12mts': [12],
        'enq_L6m': [1],
        'enq_L3m': [0],
        'Approved_Flag_P4': [0]
    })
model.predict(input_data)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([713.05160739])

In [19]:
max(df['Credit_Score'])

795

So the issue is in India CIBIL has max credit score of 900, in the data maximum is 795. My personal score is 783. This is a data issue